In [1]:
import torch
import wikipedia
import requests
import os
import textwrap
import io
from pptx import Presentation
from pptx.util import Inches
from PIL import Image
from diffusers import StableDiffusionPipeline, DiffusionPipeline
from diffusers.utils import export_to_video
from TTS.api import TTS
import numpy as np
import google.generativeai as genai
from gtts import gTTS
from pathlib import Path

# Install required libraries
# (Commented out for production; use requirements.txt)
# os.system("pip install torch wikipedia requests librosa numpy pathlib python-pptx diffusers transformers TTS pillow google-generativeai gTTS")

DEFAULT_IMAGE = "default_placeholder.jpg"

# Load Stable Diffusion model
print("Loading Stable Diffusion model...")
model_id = "stabilityai/stable-diffusion-2"
stable_diffusion = StableDiffusionPipeline.from_pretrained(model_id).to("cuda")

# Load TTS model
print("Loading TTS model...")
tts_model = TTS("tts_models/en/ljspeech/tacotron2-DDC").to("cuda")

# Load Video Diffusion pipeline
print("Loading Video Diffusion model...")
video_pipe = DiffusionPipeline.from_pretrained(
    "cerspense/zeroscope_v2_576w",
    torch_dtype=torch.float16
).to("cuda")

# Configure Gemini model
print("Loading Gemini AI...")
genai.configure(api_key="YOUR_API_KEY_HERE")  # Replace with your actual API key
gemini_model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Optional CUDA tweaks
torch.backends.cuda.matmul.allow_tf32 = True

def generate_text(prompt):
    try:
        response = gemini_model.generate_content(prompt)
        text_response = response.text
        print("📝 Generated text:\n", text_response)

        # Save and notify audio
        tts = gTTS(text_response)
        tts.save("generated_text.mp3")
        print("🔊 Response also saved as generated_text.mp3 (play manually if needed)")
    except Exception as e:
        print(f"⚠️ Gemini Error: {str(e)}")

def generate_image(prompt):
    try:
        image = stable_diffusion(prompt, num_inference_steps=35, guidance_scale=9).images[0]
        image.save("generated_image.png")
        print("🖼️ Image generated and saved as generated_image.png")
    except Exception as e:
        print(f"Failed to generate image: {e}")

def generate_audio(text):
    try:
        tts_model.tts_to_file(text=text, file_path="generated_sound.wav")
        print("🔈 Audio generated and saved as generated_sound.wav")
    except Exception as e:
        print(f"Failed to generate audio: {e}")

def generate_video(prompt):
    try:
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

        print("🎬 Generating video frames...")
        video_output = video_pipe(
            prompt=prompt,
            width=384,
            height=256,
            num_frames=24,
            num_inference_steps=25,
        ).frames

        frames = video_output[0]
        print(f"Frames shape: {frames.shape}, dtype: {frames.dtype}")

        processed_frames = []
        for frame in frames:
            frame = np.clip(frame * 255, 0, 255).astype(np.uint8)
            processed_frames.append(frame)

        export_to_video(processed_frames, "output.mp4", fps=24)
        print("🎥 Video saved as output.mp4")
    except Exception as e:
        print(f"Failed to generate video: {e}")

def get_best_wikipedia_page(topic):
    try:
        search_results = wikipedia.search(topic)
        return search_results[0] if search_results else topic
    except Exception as e:
        print(f"Error searching Wikipedia: {e}")
        return topic

def fetch_image(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic.replace(' ', '_')}"
    try:
        response = requests.get(url).json()
        if "thumbnail" in response and "source" in response["thumbnail"]:
            return response["thumbnail"]["source"]
    except Exception as e:
        print(f"Error fetching image: {e}")
    return None

def download_image(image_url):
    if not image_url:
        return DEFAULT_IMAGE
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        img = Image.open(io.BytesIO(response.content))
        img_format = img.format.lower()
        if img_format not in ["jpeg", "png"]:
            print(f"Unsupported image format: {img_format}")
            return DEFAULT_IMAGE
        img_path = f"topic_image.{img_format}"
        img.save(img_path)
        return img_path
    except Exception as e:
        print(f"Error downloading image: {e}")
        return DEFAULT_IMAGE

def generate_slide_content(topic):
    topic = get_best_wikipedia_page(topic)
    try:
        summary = wikipedia.summary(topic, sentences=10)
        wrapped_text = textwrap.wrap(summary, width=200)
        return wrapped_text
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation Error: {e.options[:5]}")
    except wikipedia.exceptions.PageError:
        print("Page not found.")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return None

def create_ppt(topic, content):
    prs = Presentation()
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = topic
    subtitle.text = "A Comprehensive Overview"

    image_url = fetch_image(topic)
    img_path = download_image(image_url)

    if img_path:
        slide.shapes.add_picture(img_path, Inches(6), Inches(1), Inches(3), Inches(2))

    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    content_box = slide.placeholders[1]
    title.text = "Introduction"
    content_box.text = f"This presentation provides an overview of {topic}."

    subtopics = ["Background", "Main Concepts", "Applications", "Advantages", "Challenges", "Future Scope"]

    for i in range(len(subtopics)):
        slide_layout = prs.slide_layouts[1]
        slide = prs.slides.add_slide(slide_layout)
        title = slide.shapes.title
        content_box = slide.placeholders[1]
        title.text = subtopics[i]
        content_box.text = content[i] if i < len(content) else "More details available online."

    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    content_box = slide.placeholders[1]
    title.text = "Conclusion"
    content_box.text = f"Thank you for exploring {topic}."

    ppt_filename = f"{topic.replace(' ', '_')}.pptx"
    prs.save(ppt_filename)
    print(f"📊 Presentation saved as {ppt_filename}")

    if img_path and img_path != DEFAULT_IMAGE:
        os.remove(img_path)

def main():
    while True:
        choice = input("\nWhat do you want to generate? (text/image/speech/ppt/video/exit): ").strip().lower()
        if choice == "exit":
            print("👋 Goodbye!")
            break
        text_input = input("Enter the text/topic: ")
        if choice == "text":
            generate_text(text_input)
        elif choice == "image":
            generate_image(text_input)
        elif choice == "speech":
            generate_audio(text_input)
        elif choice == "ppt":
            content = generate_slide_content(text_input)
            if content:
                create_ppt(text_input, content)
            else:
                print("Could not generate PPT content.")
        elif choice == "video":
            generate_video(text_input)
        else:
            print("Invalid choice. Please choose text, image, speech, ppt, video, or exit.")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'wikipedia'